In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
egg_weights = {"Common":203,"Rare":44,"Epic":7, "Legendary":1, "Manaphy":1}
egg_weights_df = pd.DataFrame(egg_weights, index = ["Weight"]).T
egg_weights_df = egg_weights_df.reset_index().rename(columns = {"index":"Type"})

#Load new epic egg weights file (thank you c4v)
epic_egg_new = pd.read_csv("epic_egg_weights_updated_fixed.csv")
epic_egg_new = epic_egg_new[:-1] #Drop the last row
epic_egg_new["p"] = epic_egg_new.loc[:,"Weight"]/epic_egg_new.loc[:,"Weight"].sum()

paradox_new = list(epic_egg_new[epic_egg_new["Label"] == "PARADOX"].iloc[:,0])
egg_only_new = list(epic_egg_new[epic_egg_new["Label"] == "EGGSCLUSIVE"].iloc[:,0])

all_egg_only_new = paradox_new + egg_only_new

In [3]:
egg_weights_df

,Type,Weight
0,Common,203
1,Rare,44
2,Epic,7
3,Legendary,1
4,Manaphy,1


In [4]:
egg_weights_df['p'] = egg_weights_df.loc[:,"Weight"]/egg_weights_df.loc[:,"Weight"].sum()

In [5]:
def choose_egg(df):
    return np.random.choice(df.iloc[:,0], p = df.loc[:,'p'])

In [6]:
def complete_eggsclusives_full(df, epic_egg_df, epic_pity = -1, legend_pity =-1, egg_targets = all_egg_only_new):
    egg_list = []
    
    cond = 0
    
    epic_counter = 0 
    legend_counter = 0
    
    epic_pity_freq = 0
    leg_pity_freq = 0
    
    #Check if egg_targets are in the random choices
    all_possible_choices = list(df.iloc[:,0]) + list(epic_egg_df.iloc[:,0])
    if set(egg_targets) <= set(all_possible_choices):
        pass
    else:
        return "Egg targets not in possible choices"
    
    while cond == False:
        epic_pity_hit = (epic_counter == epic_pity) #check if pity is hit
        leg_pity_hit = (legend_counter == legend_pity)
        
        if leg_pity_hit: #Do this check first to prioritize leg pity over the epic pity
            current_egg = "Legend"
            legend_counter = 0
            leg_pity_freq += 1
            
        elif epic_pity_hit:
            current_egg = "Epic"
            epic_counter = 0
            epic_pity_freq += 1
            
        else:
            current_egg = choose_egg(df) #Get random egg instead
        
        if (current_egg == "Common") | (current_egg == "Rare"): #Increment pity if you get common/rare
            epic_counter += 1 #increment both counters if no epic or leg
            legend_counter += 1
        
        elif current_egg == "Epic":
            current_egg = choose_egg(epic_egg_df)
            legend_counter += 1 #increment leg counter by 1
            
        elif current_egg == "Legend":
            epic_counter += 1 #Increment epic counter by 1
            
        egg_list.append(current_egg) #append obtained egg
        
        cond = all(elem in egg_list for elem in egg_targets) #Check if all egg targets are obtained
        
    return egg_list, epic_pity_freq, leg_pity_freq

In [7]:
one_vic_target, epic_pity_freq, leg_pity_freq = complete_eggsclusives_full(egg_weights_df, epic_egg_new, epic_pity = 25, legend_pity = -1, egg_targets = ["VICTINI"])

In [8]:
epic_pity_freq, len(one_vic_target)

(127, 3444)

In [9]:
def run_full_egg_sim(df, epic_egg_df, epic_pity, legend_pity, num_realizations, filename, egg_targets = all_egg_only_new, save = True):
    egg_collect = []
    epic_pity_freq = []
    leg_pity_freq = []

    for i in range(num_realizations):
        new_eggs, epic_freq, leg_freq = complete_eggsclusives_full(df, epic_egg_df, epic_pity, legend_pity, egg_targets)
        egg_collect.append(new_eggs)
        
        epic_pity_freq.append(epic_freq)
        leg_pity_freq.append(leg_freq)

    egg_counts = [len(egg_list) for egg_list in egg_collect]

    egg_realizations_new_df = pd.DataFrame([egg_collect , egg_counts, epic_pity_freq, leg_pity_freq]).T
    egg_realizations_new_df.columns = ["egg_data", "counts", "epic_pity_freq", "leg_pity_freq"]
    
    # if save:
    #     egg_realizations_new_df.to_pickle(filename) #Save DataFrame to a pickle file for storage
    # else:
    #     pass
    
    return egg_realizations_new_df

In [10]:
# full_sim_no_pity_1k = run_full_egg_sim(egg_weights_df, epic_egg_new, epic_pity = -1, legend_pity = -1, num_realizations = 1000, filename = "full_sim_no_pity_1k.pkl", egg_targets = all_egg_only_new, save = False)
# print("Done running no pity")

# full_sim_25_epic_pity_1k = run_full_egg_sim(egg_weights_df, epic_egg_new, epic_pity = 25, legend_pity = 300, num_realizations = 1000, filename = "full_sim_25_pity_1k.pkl", egg_targets = all_egg_only_new, save = True)
# print("Done running epic pity: 25")

# full_sim_72_epic_pity_1k = run_full_egg_sim(egg_weights_df, epic_egg_new, epic_pity = 72, legend_pity = 300, num_realizations = 1000, filename = "full_sim_72_pity_1k.pkl", egg_targets = all_egg_only_new, save = True)
# print("Done running epic pity: 72")

In [11]:
run_full_egg_sim(egg_weights_df, epic_egg_new, epic_pity = -1, legend_pity = -1, num_realizations = 1000, filename = "full_sim_no_pity_1k.pkl", egg_targets = all_egg_only_new, save = False)

,egg_data,counts,epic_pity_freq,leg_pity_freq
0,"[Common, Common, Common, Common, Rare, Common,...",12711,0,0
1,"[Common, Common, Common, Common, Common, Commo...",15874,0,0
2,"[Common, Rare, Common, Rare, Common, Common, R...",14785,0,0
3,"[Rare, Common, Common, Common, Common, Common,...",13802,0,0
4,"[Rare, Common, Common, Rare, Common, IRON_TREA...",13145,0,0
...,...,...,...,...
995,"[Common, Common, Rare, Common, Common, Common,...",11388,0,0
996,"[Common, Common, Common, BUZZWOLE, Common, Com...",20343,0,0
997,"[Common, Common, Common, Rare, Common, Common,...",11815,0,0
998,"[Rare, Common, Rare, Common, Common, Common, C...",20235,0,0
